In [1]:
import h2o

In [2]:
h2o.init()

H2O cluster uptime:,44 minutes 49 seconds 149 milliseconds
H2O cluster version:,3.7.0.99999
H2O cluster name:,wendycwong
H2O cluster total nodes:,1
H2O cluster total memory:,3.56 GB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [3]:
# Airlines dataset
air = h2o.import_frame(path=h2o.locate("smalldata/airlines/AirlinesTrain.csv.zip"))


Parse Progress: [##################################################] 100%


In [4]:
# Construct validation and training datasets by sampling (20/80)
r = air[0].runif()
air_train = air[r < 0.8]
air_valid = air[r >= 0.8]

myX = ["Origin", "Dest", "Distance", "UniqueCarrier", "fMonth", "fDayofMonth", "fDayOfWeek"]
myY = "IsDepDelayed"

In [5]:
# Build gbm
gbm = h2o.gbm(x=air_train[myX], 
              y=air_train[myY], 
              validation_x=air_valid[myX],
              validation_y=air_valid[myY],
              distribution="bernoulli", 
              ntrees=100, 
              max_depth=3, 
              learn_rate=0.01)


gbm Model Build Progress: [##################################################] 100%


In [6]:
# Show various confusion matrices for training dataset (based on metric(s))
print gbm.confusion_matrix() # maximum f1 threshold chosen by default

print gbm.confusion_matrix(metrics="f2")

print gbm.confusion_matrix(metrics="precision")

cms = gbm.confusion_matrix(metrics=["accuracy", "f0point5"])
print cms[0]
print cms[1]


Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.431227669022:



,NO,YES,Error,Rate
NO,2241.0,6617.0,0.747,(6617.0/8858.0)
YES,820.0,9769.0,0.0774,(820.0/10589.0)
Total,3061.0,16386.0,0.3824,(7437.0/19447.0)




Confusion Matrix (Act/Pred) for max f2 @ threshold = 0.383564166587:



,NO,YES,Error,Rate
NO,222.0,8636.0,0.9749,(8636.0/8858.0)
YES,36.0,10553.0,0.0034,(36.0/10589.0)
Total,258.0,19189.0,0.4459,(8672.0/19447.0)




Confusion Matrix (Act/Pred) for max precision @ threshold = 0.686994971965:



,NO,YES,Error,Rate
NO,8858.0,0.0,0.0,(0.0/8858.0)
YES,10583.0,6.0,0.9994,(10583.0/10589.0)
Total,19441.0,6.0,0.5442,(10583.0/19447.0)




Confusion Matrix (Act/Pred) for max accuracy @ threshold = 0.530485784553:



,NO,YES,Error,Rate
NO,5127.0,3731.0,0.4212,(3731.0/8858.0)
YES,3025.0,7564.0,0.2857,(3025.0/10589.0)
Total,8152.0,11295.0,0.3474,(6756.0/19447.0)




Confusion Matrix (Act/Pred) for max f0point5 @ threshold = 0.545185232938:



,NO,YES,Error,Rate
NO,5495.0,3363.0,0.3797,(3363.0/8858.0)
YES,3418.0,7171.0,0.3228,(3418.0/10589.0)
Total,8913.0,10534.0,0.3487,(6781.0/19447.0)


In [7]:
# Show various confusion matrices for training dataset (based on threshold(s))
print gbm.confusion_matrix(thresholds=0.77) 

cms = gbm.confusion_matrix(thresholds=[0.1, 0.5, 0.99])
print cms[0]
print cms[1]
print cms[2]

Could not find exact threshold 0.77; using closest threshold found 0.686994971965.

Confusion Matrix (Act/Pred) @ threshold = 0.686994971965:



,NO,YES,Error,Rate
NO,8858.0,0.0,0.0,(0.0/8858.0)
YES,10583.0,6.0,0.9994,(10583.0/10589.0)
Total,19441.0,6.0,0.5442,(10583.0/19447.0)



Could not find exact threshold 0.1; using closest threshold found 0.379467159837.
Could not find exact threshold 0.5; using closest threshold found 0.499997150547.
Could not find exact threshold 0.99; using closest threshold found 0.686994971965.

Confusion Matrix (Act/Pred) @ threshold = 0.379467159837:



,NO,YES,Error,Rate
NO,0.0,8858.0,1.0,(8858.0/8858.0)
YES,0.0,10589.0,0.0,(0.0/10589.0)
Total,0.0,19447.0,0.4555,(8858.0/19447.0)




Confusion Matrix (Act/Pred) @ threshold = 0.499997150547:



,NO,YES,Error,Rate
NO,4449.0,4409.0,0.4977,(4409.0/8858.0)
YES,2379.0,8210.0,0.2247,(2379.0/10589.0)
Total,6828.0,12619.0,0.3491,(6788.0/19447.0)




Confusion Matrix (Act/Pred) @ threshold = 0.686994971965:



,NO,YES,Error,Rate
NO,8858.0,0.0,0.0,(0.0/8858.0)
YES,10583.0,6.0,0.9994,(10583.0/10589.0)
Total,19441.0,6.0,0.5442,(10583.0/19447.0)


In [8]:
# Show various confusion matrices for validation dataset (based on metric(s))
print gbm.confusion_matrix(metrics="f2", valid=True)

print gbm.confusion_matrix(metrics="precision", valid=True)

cms = gbm.confusion_matrix(metrics=["accuracy", "f0point5"], valid=True)
print cms[0]
print cms[1]


Confusion Matrix (Act/Pred) for max f2 @ threshold = 0.386095698558:



,NO,YES,Error,Rate
NO,87.0,2121.0,0.9606,(2121.0/2208.0)
YES,13.0,2753.0,0.0047,(13.0/2766.0)
Total,100.0,4874.0,0.429,(2134.0/4974.0)




Confusion Matrix (Act/Pred) for max precision @ threshold = 0.686994962601:



,NO,YES,Error,Rate
NO,2208.0,0.0,0.0,(0.0/2208.0)
YES,2765.0,1.0,0.9996,(2765.0/2766.0)
Total,4973.0,1.0,0.5559,(2765.0/4974.0)




Confusion Matrix (Act/Pred) for max accuracy @ threshold = 0.510841362146:



,NO,YES,Error,Rate
NO,1175.0,1033.0,0.4678,(1033.0/2208.0)
YES,707.0,2059.0,0.2556,(707.0/2766.0)
Total,1882.0,3092.0,0.3498,(1740.0/4974.0)




Confusion Matrix (Act/Pred) for max f0point5 @ threshold = 0.547964648642:



,NO,YES,Error,Rate
NO,1362.0,846.0,0.3832,(846.0/2208.0)
YES,917.0,1849.0,0.3315,(917.0/2766.0)
Total,2279.0,2695.0,0.3544,(1763.0/4974.0)


In [9]:
# Show various confusion matrices for validation dataset (based on threshold(s))
print gbm.confusion_matrix(thresholds=0.77) 

cms = gbm.confusion_matrix(thresholds=[0.25, 0.33, 0.44])
print cms[0]
print cms[1]
print cms[2]

Could not find exact threshold 0.77; using closest threshold found 0.686994971965.

Confusion Matrix (Act/Pred) @ threshold = 0.686994971965:



,NO,YES,Error,Rate
NO,8858.0,0.0,0.0,(0.0/8858.0)
YES,10583.0,6.0,0.9994,(10583.0/10589.0)
Total,19441.0,6.0,0.5442,(10583.0/19447.0)



Could not find exact threshold 0.25; using closest threshold found 0.379467159837.
Could not find exact threshold 0.33; using closest threshold found 0.379467159837.
Could not find exact threshold 0.44; using closest threshold found 0.441443306103.

Confusion Matrix (Act/Pred) @ threshold = 0.379467159837:



,NO,YES,Error,Rate
NO,0.0,8858.0,1.0,(8858.0/8858.0)
YES,0.0,10589.0,0.0,(0.0/10589.0)
Total,0.0,19447.0,0.4555,(8858.0/19447.0)




Confusion Matrix (Act/Pred) @ threshold = 0.379467159837:



,NO,YES,Error,Rate
NO,0.0,8858.0,1.0,(8858.0/8858.0)
YES,0.0,10589.0,0.0,(0.0/10589.0)
Total,0.0,19447.0,0.4555,(8858.0/19447.0)




Confusion Matrix (Act/Pred) @ threshold = 0.441443306103:



,NO,YES,Error,Rate
NO,2555.0,6303.0,0.7116,(6303.0/8858.0)
YES,1014.0,9575.0,0.0958,(1014.0/10589.0)
Total,3569.0,15878.0,0.3763,(7317.0/19447.0)


In [10]:
# Show various confusion matrices for validation dataset (based on metric(s) AND threshold(s))
cms = gbm.confusion_matrix(thresholds=0.77, metrics="f1") 
print cms[0]
print cms[1]

cms = gbm.confusion_matrix(thresholds=[0.25, 0.33], metrics=["f2", "f0point5"])
print cms[0]
print cms[1]
print cms[2]
print cms[3]

Could not find exact threshold 0.77; using closest threshold found 0.686994971965.

Confusion Matrix (Act/Pred) @ threshold = 0.686994971965:



,NO,YES,Error,Rate
NO,8858.0,0.0,0.0,(0.0/8858.0)
YES,10583.0,6.0,0.9994,(10583.0/10589.0)
Total,19441.0,6.0,0.5442,(10583.0/19447.0)




Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.431227669022:



,NO,YES,Error,Rate
NO,2241.0,6617.0,0.747,(6617.0/8858.0)
YES,820.0,9769.0,0.0774,(820.0/10589.0)
Total,3061.0,16386.0,0.3824,(7437.0/19447.0)



Could not find exact threshold 0.25; using closest threshold found 0.379467159837.
Could not find exact threshold 0.33; using closest threshold found 0.379467159837.

Confusion Matrix (Act/Pred) @ threshold = 0.379467159837:



...

,NO,YES,Error,Rate
NO,0.0,8858.0,1.0,(8858.0/8858.0)
YES,0.0,10589.0,0.0,(0.0/10589.0)
Total,0.0,19447.0,0.4555,(8858.0/19447.0)




Confusion Matrix (Act/Pred) for max f2 @ threshold = 0.383564166587:



,NO,YES,Error,Rate
NO,222.0,8636.0,0.9749,(8636.0/8858.0)
YES,36.0,10553.0,0.0034,(36.0/10589.0)
Total,258.0,19189.0,0.4459,(8672.0/19447.0)




Confusion Matrix (Act/Pred) for max f0point5 @ threshold = 0.545185232938:



,NO,YES,Error,Rate
NO,5495.0,3363.0,0.3797,(3363.0/8858.0)
YES,3418.0,7171.0,0.3228,(3418.0/10589.0)
Total,8913.0,10534.0,0.3487,(6781.0/19447.0)


In [11]:
# Test dataset
air_test = h2o.import_frame(path=h2o.locate("smalldata/airlines/AirlinesTest.csv.zip"))

# Test performance
gbm_perf = gbm.model_performance(air_test)


Parse Progress: [##################################################] 100%


In [12]:
# Show various confusion matrices for test dataset (based on metric(s))
print gbm_perf.confusion_matrix(metrics="f0point5")

print gbm_perf.confusion_matrix(metrics="min_per_class_accuracy")

cms = gbm_perf.confusion_matrix(metrics=["accuracy", "f0point5"])
print cms[0]
print cms[1]


Confusion Matrix (Act/Pred) for max f0point5 @ threshold = 0.547980890324:



,NO,YES,Error,Rate
NO,780.0,437.0,0.3591,(437.0/1217.0)
YES,483.0,991.0,0.3277,(483.0/1474.0)
Total,1263.0,1428.0,0.3419,(920.0/2691.0)




Confusion Matrix (Act/Pred) for max min_per_class_accuracy @ threshold = 0.548759203974:



,NO,YES,Error,Rate
NO,791.0,426.0,0.35,(426.0/1217.0)
YES,525.0,949.0,0.3562,(525.0/1474.0)
Total,1316.0,1375.0,0.3534,(951.0/2691.0)




Confusion Matrix (Act/Pred) for max accuracy @ threshold = 0.547980890324:



,NO,YES,Error,Rate
NO,780.0,437.0,0.3591,(437.0/1217.0)
YES,483.0,991.0,0.3277,(483.0/1474.0)
Total,1263.0,1428.0,0.3419,(920.0/2691.0)




Confusion Matrix (Act/Pred) for max accuracy @ threshold = 0.547980890324:



,NO,YES,Error,Rate
NO,780.0,437.0,0.3591,(437.0/1217.0)
YES,483.0,991.0,0.3277,(483.0/1474.0)
Total,1263.0,1428.0,0.3419,(920.0/2691.0)


In [13]:
# Show various confusion matrices for test dataset (based on threshold(s))
print gbm_perf.confusion_matrix(thresholds=0.5) 

cms = gbm_perf.confusion_matrix(thresholds=[0.01, 0.75, .88])
print cms[0]
print cms[1]
print cms[2]

Could not find exact threshold 0.5; using closest threshold found 0.500038337835.

Confusion Matrix (Act/Pred) @ threshold = 0.500038337835:



,NO,YES,Error,Rate
NO,577.0,640.0,0.5259,(640.0/1217.0)
YES,312.0,1162.0,0.2117,(312.0/1474.0)
Total,889.0,1802.0,0.3538,(952.0/2691.0)



Could not find exact threshold 0.01; using closest threshold found 0.379467146787.
Could not find exact threshold 0.75; using closest threshold found 0.685667942575.
Could not find exact threshold 0.88; using closest threshold found 0.685667942575.

Confusion Matrix (Act/Pred) @ threshold = 0.379467146787:



,NO,YES,Error,Rate
NO,0.0,1217.0,1.0,(1217.0/1217.0)
YES,0.0,1474.0,0.0,(0.0/1474.0)
Total,0.0,2691.0,0.4522,(1217.0/2691.0)




Confusion Matrix (Act/Pred) @ threshold = 0.685667942575:



,NO,YES,Error,Rate
NO,1216.0,1.0,0.0008,(1.0/1217.0)
YES,1472.0,2.0,0.9986,(1472.0/1474.0)
Total,2688.0,3.0,0.5474,(1473.0/2691.0)




Confusion Matrix (Act/Pred) @ threshold = 0.685667942575:



,NO,YES,Error,Rate
NO,1216.0,1.0,0.0008,(1.0/1217.0)
YES,1472.0,2.0,0.9986,(1472.0/1474.0)
Total,2688.0,3.0,0.5474,(1473.0/2691.0)


In [14]:
# Convert a ConfusionMatrix to a python list of lists: [ [tns,fps], [fns,tps] ]
cm = gbm.confusion_matrix()
print cm.to_list()

cm = gbm_perf.confusion_matrix()
print cm.to_list()

[[2241, 6617], [820, 9769]]
[[296, 921], [114, 1360]]
